# Advanced Databases Course project

## Analysis data and creating database contaning information about FIFA computer game cards.

![image](assets/fifa_logo.jpeg)

## Fifa has online game mode, where user collects the cards. Those cards are used to create own club. Each card includes information about player: personal information as well as technical skills of each player.

<img src="assets/robercik.jpg" width="300" height="600" align="center"/>

## This project tries to analize all the information included in cards and creates a SQL database which contain all the analysed information.

In [1]:
import pandas as pd

In [2]:
DATA_PATH = "data/players_20.csv"

In [3]:
df  = pd.read_csv(DATA_PATH)

# Preparing the data

## Does the data was successfully loaded?

In [4]:
df.head(2)

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,...,68+2,66+2,66+2,66+2,68+2,63+2,52+2,52+2,52+2,63+2
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,...,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3


## Reading unique information

In [5]:
clubs = df["club"].unique()

In [6]:
national_teams = df["nationality"].unique()

In [7]:
body_types = df["body_type"].unique()

In [8]:
work_rates = df["work_rate"].unique()

In [9]:
player_traits_lines= df["player_traits"].unique()
player_traits = []
for trait_line in player_traits_lines:
    player_traits += [trait.strip() for trait in str(trait_line).split(",")]

In [10]:
player_tags_lines = df["player_tags"].unique()
player_tags = []
for tag_line_line in player_tags_lines:
    player_tags += [str(tag.strip()[1:]) for tag in str(tag_line_line).split(",")]

In [11]:
player_positions_lines = df["player_positions"].unique()
player_positions = []
for position_line in player_positions_lines:
    player_positions += [str(position.strip()) for position in str(position_line).split(",")]
    
player_positions+= df["nation_position"].dropna().unique().tolist()
player_positions+= df["team_position"].dropna().unique().tolist()
player_positions = list(set(player_positions))


# Validating data

## Validating columns with values between 0-100

In [12]:
between_0_100_columns = [ 'overall',
 'potential',
 'pace',
 'shooting',
 'passing',
 'dribbling',
 'defending',
 'physic',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_reflexes',
 'gk_speed',
 'gk_positioning',
 'attacking_crossing',
 'attacking_finishing',
 'attacking_heading_accuracy',
 'attacking_short_passing',
 'attacking_volleys',
 'skill_dribbling',
 'skill_curve',
 'skill_fk_accuracy',
 'skill_long_passing',
 'skill_ball_control',
 'movement_acceleration',
 'movement_sprint_speed',
 'movement_agility',
 'movement_reactions',
 'movement_balance',
 'power_shot_power',
 'power_jumping',
 'power_stamina',
 'power_strength',
 'power_long_shots',
 'mentality_aggression',
 'mentality_interceptions',
 'mentality_positioning',
 'mentality_vision',
 'mentality_penalties',
 'mentality_composure',
 'defending_marking',
 'defending_standing_tackle',
 'defending_sliding_tackle',
 'goalkeeping_diving',
 'goalkeeping_handling',
 'goalkeeping_kicking',
 'goalkeeping_positioning',
 'team_jersey_number',
 'nation_jersey_number'                     ]

### Some of the statistics are only valid for goalkeeper or filed player. In this case the value in column is just empty.This scenarion is perfectly fine. To ignore these values in the comparing, Nan columns will be changed to 99


In [13]:
df_between_0_and_100 = df[between_0_100_columns].fillna(99)
ge100_0_100 = (df_between_0_and_100.ge(100)).any().any()
le0_0_100 = (df_between_0_and_100.le(0)).any().any()
valid0_100 = (not ge100_0_100) and (not le0_0_100) 

# Validating columns with values between 0-5

In [14]:
df_between_0_and_5 = ['international_reputation',
 'weak_foot',
 'skill_moves']

df_between_0_and_5 = df[df_between_0_and_5]
ge5_0_5 = (df_between_0_and_5.ge(5)).any().any()
le0_0_5 = (df_between_0_and_5.le(0)).any().any()
valid0_5 = (not le0_0_5) and (not ge5_0_5) 

# Conecting with database

In [15]:
import sys
!{sys.executable} -m pip install SQLAlchemy
!{sys.executable} -m pip install psycopg2-binary

In [16]:
from sqlalchemy import create_engine, MetaData, Table,ForeignKey, Sequence, CheckConstraint, UniqueConstraint, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Date
from sqlalchemy.dialects.mysql import INTEGER

conn_url = 'postgresql+psycopg2://mcichock:student@db/fifa'
engine = create_engine(conn_url)
Base = declarative_base()

## Creating tables

### Defining Enums

In [17]:
from sqlalchemy import Enum
import enum

class BetterFoot(enum.Enum):
    LEFT = 0
    RIGHT = 1

### BodyType

In [18]:
class BodyType(Base):
    __tablename__ = 'BodyType'
    __table_args__ = (    
        UniqueConstraint('body_type'),
    )
    id = Column(Integer, primary_key=True)
    body_type = Column(String(50), nullable=False, server_default="Normal")
    
    def __repr__(self):
        return "<BodyType(id='{0}', body_type={1}".format(
            self.id, self.body_type)

### Work Rate

In [19]:
class WorkRate(Base):
    __tablename__ = 'WorkRate'
    __table_args__ = (    
        UniqueConstraint('work_rate'),
    )
    id = Column(Integer, primary_key=True)
    work_rate = Column(String(50), nullable=False)

    def __repr__(self):
        return "<WorkRate(id='{0}', work_rate={1}".format(
            self.id, self.work_rate)

### PersonalPhysics

In [20]:
class PersonalPhysical(Base):
    __tablename__ = 'PersonalPhysical'
    id = Column(Integer, primary_key=True)
    height = Column(INTEGER(unsigned=True), nullable=False)
    weight = Column(INTEGER(unsigned=True), nullable=False)
    body_type_id = Column(Integer, ForeignKey('BodyType.id'))
    
    
    def __repr__(self):
        return "<PersonalPhysical(id='{0}', height={1}, weight={2}, body_type_id={3}".format(
            self.id, self.height, self.weight,self.body_type_id)

### PersonalInfo

In [21]:
class PersonalInfo(Base):
    __tablename__ = 'PersonalInfo'
    id = Column(Integer, primary_key=True)
    short_name = Column(String(60), nullable=False)
    long_name = Column(String(300), nullable=False)
    dob = Column(Date, nullable=False)
    nationality = Column(String(60), nullable=False)
    def __repr__(self):
        return "<PersonalInfo(id='{0}', short_name={1}, long_name={2}, dob={3}, nationality={4}".format(
            self.id, self.short_name, self.long_name,self.dob, self.nationality)

### Club

In [22]:
class Club(Base):
    __tablename__ = 'Club'
    __table_args__ = (    
        UniqueConstraint('name'),
    )
    id = Column(Integer, primary_key=True)
    name = Column(String(60), nullable=False)
    def __repr__(self):
        return "<Club(id='{0}', name={1}".format(self.id, self.name)

### Contract Details

In [23]:
class ContractDetails(Base):
    __tablename__ = 'ContractDetails'
    id = Column(Integer, primary_key=True)
    wag_eur = Column(INTEGER(unsigned=True), nullable=False)
    release_eur = Column(INTEGER(unsigned=True), nullable=True)
    valid_until = Column(INTEGER(unsigned=True), nullable=False)
    loaded_from = Column(Integer, ForeignKey('Club.id'), nullable=True)
    def __repr__(self):
        return "<ContractDetails(id='{0}', wag_eur={1}, release_eur={2},valid_until={3}, loaded_from={4}".format(self.id, self.wag_eur, self.release_eur, self.valid_until, self.loaded_from)

### Nationality

In [24]:
class Nationality(Base):
    __tablename__ = 'Nationality'
    __table_args__ = (    
        UniqueConstraint('name'),
    )
    id = Column(Integer, primary_key=True)
    name = Column(String(60), nullable=False)
    def __repr__(self):
        return "<Nationality(id='{0}', name={1}".format(self.id, self.name)

### Position

In [25]:
class Position(Base):
    __tablename__ = 'Position'
    __table_args__ = (    
        UniqueConstraint('position'),
    )
    id = Column(Integer, primary_key=True)
    position = Column(String(60), nullable=False)
    def __repr__(self):
        return "<Position(id='{0}', position={1}".format(self.id, self.position)

### Trait

In [26]:
class Trait(Base):
    __tablename__ = 'Trait'
    __table_args__ = (    
        UniqueConstraint('trait'),
    )
    id = Column(Integer, primary_key=True)
    trait = Column(String(60), nullable=False)
    def __repr__(self):
        return "<Trait(id='{0}', trait={1}".format(self.id, self.trait)

### Tag

In [27]:
class Tag(Base):
    __tablename__ = 'Tag'
    __table_args__ = (    
        UniqueConstraint('tag'),
    )
    id = Column(Integer, primary_key=True)
    tag = Column(String(60), nullable=False)
    def __repr__(self):
        return "<Tag(id='{0}', tag={1}".format(self.id, self.tag)

### Attacking Stats

In [28]:
class AttackingStats(Base):
    __tablename__ = 'AttackingStats'
    __table_args__ = (    
        CheckConstraint('(crossing > 0) and (crossing <100)'),
        CheckConstraint('(finishing > 0) and (finishing <100)'),
        CheckConstraint('(heading > 0) and (heading <100)'),
        CheckConstraint('(short_pass > 0) and (short_pass <100)'),
        CheckConstraint('(volleys > 0) and (volleys <100)'),
    )
    id = Column(Integer, primary_key=True)
    crossing = Column(INTEGER(unsigned=True), nullable=False)
    finishing = Column(INTEGER(unsigned=True), nullable=False)
    heading = Column(INTEGER(unsigned=True), nullable=False)
    short_pass = Column(INTEGER(unsigned=True), nullable=False)
    volleys = Column(INTEGER(unsigned=True), nullable=False)
    def __repr__(self):
        return "<AttackingStats(id='{0}', crossing={1}, finishing={2}\
        heading={3}, short_pases-{4}, volleys={5}".format(self.id, self.crossing,\
        self.finishing.self.heading, self.short_pass. self.volleys)

### Skill Stats

In [29]:
class SkillStats(Base):
    __tablename__ = 'SkillStats'
    id = Column(Integer, primary_key=True)
    dribbling = Column(INTEGER(unsigned=True), nullable=False)
    curve = Column(INTEGER(unsigned=True), nullable=False)
    fk_accuracy = Column(INTEGER(unsigned=True), nullable=False)
    long_passing = Column(INTEGER(unsigned=True), nullable=False)
    ball_control = Column(INTEGER(unsigned=True), nullable=False)
    def __repr__(self):
        return "<SkillStats(id='{0}', dribbling={1}, curve={2}\
        fk_accuracy={3}, long_passing={4}, ball_control={5}".format(self.id, self.dribbling,\
        self.curve, self.fk_accuracy. self.long_passing. self.ball_control)

### Movement Stats

In [30]:
class MovementStats(Base):
    __tablename__ = 'MovementStats'
    __table_args__ = (    
        CheckConstraint('(acceleration > 0) and (acceleration <100)'),
        CheckConstraint('(sprint > 0) and (sprint <100)'),
        CheckConstraint('(agility > 0) and (agility <100)'),
        CheckConstraint('(reactions > 0) and (reactions <100)'),
        CheckConstraint('(balance > 0) and (balance <100)'),
    )
    id = Column(Integer, primary_key=True)
    acceleration = Column(INTEGER(unsigned=True), nullable=False)
    sprint = Column(INTEGER(unsigned=True), nullable=False)
    agility = Column(INTEGER(unsigned=True), nullable=False)
    reactions = Column(INTEGER(unsigned=True), nullable=False)
    balance = Column(INTEGER(unsigned=True), nullable=False)
    def __repr__(self):
        return "<MovementStats(id='{0}', acceleration={1}, sprint={2}\
        agility={3}, reactions-{4}, balance={5}".format(self.id, self.acceleration,\
        self.sprint, self.agility, self.reactions, self.balance)

### Power Stats

In [31]:
class PowerStats(Base):
    __tablename__ = 'PowerStats'
    __table_args__ = (    
        CheckConstraint('(shot > 0) and (shot <100)'),
        CheckConstraint('(jumping > 0) and (jumping <100)'),
        CheckConstraint('(stamina > 0) and (stamina <100)'),
        CheckConstraint('(strength > 0) and (strength <100)'),
        CheckConstraint('(long_shot > 0) and (long_shot <100)'),
    )
    id = Column(Integer, primary_key=True)
    shot = Column(INTEGER(unsigned=True), nullable=False)
    jumping = Column(INTEGER(unsigned=True), nullable=False)
    stamina = Column(INTEGER(unsigned=True), nullable=False)
    strength = Column(INTEGER(unsigned=True), nullable=False)
    long_shot = Column(INTEGER(unsigned=True), nullable=False)
    def __repr__(self):
        return "<PowerStats(id='{0}', shot={1}, jumping={2}\
        stamina={3}, strength={4}, long_shot={5}".format(self.id, self.shot,\
        self.jumping, self.stamina, self.strength, self.long_shot)

### Mentality Stats

In [32]:
class MentalityStats(Base):
    __tablename__ = 'MentalityStats'
    __table_args__ = (    
        CheckConstraint('(aggression > 0) and (aggression <100)'),
        CheckConstraint('(interceptions > 0) and (interceptions <100)'),
        CheckConstraint('(positioning > 0) and (positioning <100)'),
        CheckConstraint('(vision > 0) and (vision <100)'),
        CheckConstraint('(penalties > 0) and (penalties <100)'),
        CheckConstraint('(composure > 0) and (composure <100)'),
    )
    id = Column(Integer, primary_key=True)
    aggression = Column(INTEGER(unsigned=True), nullable=False)
    interceptions = Column(INTEGER(unsigned=True), nullable=False)
    positioning = Column(INTEGER(unsigned=True), nullable=False)
    vision = Column(INTEGER(unsigned=True), nullable=False)
    penalties = Column(INTEGER(unsigned=True), nullable=False)
    composure = Column(INTEGER(unsigned=True), nullable=False)
    def __repr__(self):
        return "<MentalityStats(id='{0}', aggression={1}, interceptions={2}\
        positioning={3}, vision={4}, penalties={5}, composure={composure}".format(self.id, self.shot,\
        self.jumping, self.stamina, self.strength, self.long_shot, self.composure)

### Defending Stats

In [33]:
class DefendingStats(Base):
    __tablename__ = 'DefendingStats'
    __table_args__ = (    
        CheckConstraint('(marking > 0) and (marking <100)'),
        CheckConstraint('(standing_tackle > 0) and (standing_tackle <100)'),
        CheckConstraint('(sliding_tackle > 0) and (sliding_tackle <100)'),
    )
    id = Column(Integer, primary_key=True)
    marking = Column(INTEGER(unsigned=True), nullable=False)
    standing_tackle = Column(INTEGER(unsigned=True), nullable=False)
    sliding_tackle = Column(INTEGER(unsigned=True), nullable=False)
    def __repr__(self):
        return "<DefendingStats(id='{0}', marking={1}, standing_tackle={2}\
        sliding_tackle={3}".format(self.id, self.marking,\
        self.standing_tackle, self.sliding_tackle)

### Goalkeeping Stats

In [34]:
class GoalkeepingStats(Base):
    __tablename__ = 'GoalkeepingStats'
    __table_args__ = (    
        CheckConstraint('(diving > 0) and (diving <100)'),
        CheckConstraint('(handling > 0) and (handling <100)'),
        CheckConstraint('(kicking > 0) and (kicking <100)'),
        CheckConstraint('(positioning > 0) and (positioning <100)'),
        CheckConstraint('(reflexes > 0) and (reflexes <100)'),
    )
    id = Column(Integer, primary_key=True)
    diving = Column(INTEGER(unsigned=True), nullable=False)
    handling = Column(INTEGER(unsigned=True), nullable=False)
    kicking = Column(INTEGER(unsigned=True), nullable=False)
    positioning = Column(INTEGER(unsigned=True), nullable=False)
    reflexes = Column(INTEGER(unsigned=True), nullable=False)
    def __repr__(self):
        return "<GoalkeepingStats(id='{0}', diving={1}, handling={2}\
        kicking={3}, positioning={4}, reflexes={5}".format(self.id, self.diving,\
        self.handling, self.kicking, self.positioning, self.reflexes)

### Detailed Stats

In [35]:
class DetailedStats(Base):
    __tablename__ = 'DetailedStats'
    id = Column(Integer, primary_key=True)
    attacking_id = Column(Integer, ForeignKey('AttackingStats.id'))
    skills_id = Column(Integer, ForeignKey('SkillStats.id'))
    movement_id = Column(Integer, ForeignKey('MovementStats.id'))
    power_id = Column(Integer, ForeignKey('PowerStats.id'))
    mentality_id =  Column(Integer, ForeignKey('MentalityStats.id'))
    defending_id =  Column(Integer, ForeignKey('DefendingStats.id'))
    goalkeeping_id =  Column(Integer, ForeignKey('GoalkeepingStats.id'))
    def __repr__(self):
        return "<DetailedStats(id='{0}', attacking_id={1}, skills_id={2}\
        movement_id={3}, power_id={4}, mentality_id={5} defending_id={6}, goalkeeping_id{7}".format(self.id, self.attacking_id,\
       self.skills_id,self.movement_id, self.power_id, self.mentality_id, self.defending_id, self.goalkeeping_id)

### Filed Player Stats

In [36]:
class FieldPlayerStats(Base):
    __tablename__ = 'FieldPlayerStats'
    __table_args__ = (    
        CheckConstraint('(pace > 0) and (pace <100)'),
        CheckConstraint('(shooting > 0) and (shooting <100)'),
        CheckConstraint('(passing > 0) and (passing <100)'),
        CheckConstraint('(dribbling > 0) and (dribbling <100)'),
        CheckConstraint('(defending > 0) and (defending <100)'),
        CheckConstraint('(physic > 0) and (physic <100)'),
    )
    id = Column(Integer, primary_key=True)
    pace= Column(INTEGER(unsigned=True), nullable=False)
    shooting = Column(INTEGER(unsigned=True), nullable=False)
    passing = Column(INTEGER(unsigned=True), nullable=False)
    dribbling = Column(INTEGER(unsigned=True), nullable=False)
    defending = Column(INTEGER(unsigned=True), nullable=False)
    physic = Column(INTEGER(unsigned=True), nullable=False)
    personal_physical_id = Column(Integer, ForeignKey('GoalkeeperStats.id'),  nullable=True)
    
    def __repr__(self):
        return "<FieldPlayerStats(id='{0}',pace={1}, shooting={2}, passing={3}\
        dribbling={4}, defending={5}, physic={6}".format(self.id, self.pace,\
        self.shooting, self.passing, self.dribbling, self.defending, self.physic)

### Goalkeer Stats

In [37]:
class GoalkeeperStats(Base):
    __tablename__ = 'GoalkeeperStats'
    __table_args__ = (    
        CheckConstraint('(diving > 0) and (diving <100)'),
        CheckConstraint('(handling > 0) and (handling <100)'),
        CheckConstraint('(kicking > 0) and (kicking <100)'),
        CheckConstraint('(reflexes > 0) and (reflexes <100)'),
        CheckConstraint('(speed > 0) and (speed <100)'),
        CheckConstraint('(positioning > 0) and (positioning <100)'),
    )
    id = Column(Integer, primary_key=True)
    diving= Column(INTEGER(unsigned=True), nullable=False)
    handling = Column(INTEGER(unsigned=True), nullable=False)
    kicking = Column(INTEGER(unsigned=True), nullable=False)
    reflexes = Column(INTEGER(unsigned=True), nullable=False)
    speed = Column(INTEGER(unsigned=True), nullable=False)
    positioning = Column(INTEGER(unsigned=True), nullable=False)
    def __repr__(self):
        return "<GoalkeeperStats(id='{0}',diving={1}, handling={2}, kicking={3}\
        reflexes={4}, speed={5}, positioning={6}".format(self.id, self.diving,\
        self.handling, self.kicking, self.reflexes, self.speed, self.positioning)

### Overall Stats

In [38]:
class OverallStats(Base):
    __tablename__ = 'OverallStats'
    id = Column(Integer, primary_key=True)
    overall= Column(INTEGER(unsigned=True), nullable=False)
    potential = Column(INTEGER(unsigned=True), nullable=False)
    field_player_id = Column(Integer, ForeignKey('FieldPlayerStats.id'),  nullable=True)
    goalkeeper_id = Column(Integer, ForeignKey('GoalkeeperStats.id'),  nullable=True)
    def __repr__(self):
        return "<OverallStats(id='{0}', overall={1}, potential={2}\
        field_player_id={3}, goalkeeper_id={4}".format(self.id, self.overall,\
       self.potential, self.field_player_id, self.goalkeeper_id)

### Stars

In [39]:
class Stars(Base):
    __tablename__ = 'Stars'
    __table_args__ = (    
        CheckConstraint('(skills > 0) and (skills <6)'),
        CheckConstraint('(reputation > 0) and (reputation <6)'),
        CheckConstraint('(weak_foot > 0) and (weak_foot <6)'),
    )
    id = Column(Integer, primary_key=True)
    skills= Column(INTEGER(unsigned=True), nullable=False)
    reputation = Column(INTEGER(unsigned=True), nullable=False)
    weak_foot = Column(INTEGER(unsigned=True), nullable=False)
    def __repr__(self):
        return "<Stars(id='{0}',skills={1}, reputation={2}, weak_foot={3}".format(self.id, self.skills,\
        self.reputation, self.weak_foot)

 ### Fifa Card

In [40]:
class FifaCard(Base):
    __tablename__ = 'FifaCard'

    id = Column(Integer, primary_key=True)
    personal_info_id = Column(Integer, ForeignKey('PersonalInfo.id'),  nullable=False)
    personal_physical_id = Column(Integer, ForeignKey('PersonalPhysical.id'), nullable=False)
    overall_stats_id = Column(Integer, ForeignKey('OverallStats.id'),  nullable=False)
    detailed_stats_id = Column(Integer, ForeignKey('DetailedStats.id'),  nullable=False)
    nationality_id = Column(Integer, ForeignKey('Nationality.id'),  nullable=False)
    work_rate_id = Column(Integer, ForeignKey('WorkRate.id'),  nullable=False)
    real_face = Column(Boolean, default=True,  nullable=False)
    better_foot = Column(Enum(BetterFoot), default=BetterFoot.RIGHT)
    value_eur = Column(INTEGER(unsigned=True),  nullable=False)
    
    def __repr__(self):
        return "<FifaCard(id='{0}', personal_info_id={1}, overall_stats_id={2}\
        detailed_stats_id={3}, work_rate_id={4}, real_face={5}, better_foot={6}, personal_physical_id={7}, value_eur={8}, nationality_id={7}".format(self.id, self.personal_info_id,\
       self.overall_stats_id, self.detailed_stats_id, self.work_rate_id, self.real_face, self.better_foot, self.personal_physical_id, self.value_eur, self.nationality_id)

### Fifa Card To Position

In [41]:
class PositionToFifaCard(Base):
    __tablename__ = 'PositionToFifaCard'
    id = Column(Integer, primary_key=True)
    card_id = Column(Integer, ForeignKey('FifaCard.id'),  nullable=False)
    position_id = Column(Integer, ForeignKey('Position.id'),  nullable=False)
    
    def __repr__(self):
        return "<PositionToFifaCard(id='{0}', card_id={1}, position_id={2}".format(self.id, self.card_id,\
       self.position_id)

### Trait To Fifa Card

In [42]:
class TraitToFifaCard(Base):
    __tablename__ = 'TraitToFifaCard'
    id = Column(Integer, primary_key=True)
    card_id = Column(Integer, ForeignKey('FifaCard.id'),  nullable=False)
    trait_id = Column(Integer, ForeignKey('Trait.id'),  nullable=False)
    
    def __repr__(self):
        return "<TraitToFifaCard(id='{0}', card_id={1}, trait_id={2}".format(self.id, self.card_id,\
       self.trait_id)

### Tag To Fifa Card

In [43]:
class TagToFifaCard(Base):
    __tablename__ = 'TagToFifaCard'
    id = Column(Integer, primary_key=True)
    card_id = Column(Integer, ForeignKey('FifaCard.id'),  nullable=False)
    tag_id = Column(Integer, ForeignKey('Tag.id'),  nullable=False)
    
    def __repr__(self):
        return "<TagToFifaCard(id='{0}', card_id={1}, tag_id={2}".format(self.id, self.card_id,\
       self.tag_id)

### Card in Club

In [44]:
class CardInClub(Base):
    __tablename__ = 'CardInClub'
    __table_args__ = (
        CheckConstraint('(number > 0) and (number <100)'),
    )
    id = Column(Integer, primary_key=True)
    card_id = Column(Integer, ForeignKey('FifaCard.id'),  nullable=False)
    number = Column(INTEGER(unsigned=True),  nullable=True)
    joined = Column(Date,  nullable=True)
    position = Column(Integer, ForeignKey('Position.id'),  nullable=True)
    contract_details_id = Column(Integer, ForeignKey('ContractDetails.id'),  nullable=False)
    club_id = Column(Integer, ForeignKey('Club.id'),  nullable=False)
    
    def __repr__(self):
        return "<CardInClub(id='{0}', card_id={1}, number={2}, joined={3}, position={4},contract_detail={5}".format(self.id, self.card_id,\
       self.number, self.joined, self.joined,self.position, self.contract_details_id)

### Card in National Team

In [45]:
class CardInNationalTeam(Base):
    __tablename__ = 'CardInNationalTeam'
    __table_args__ = (
        CheckConstraint('(number > 0) and (number <100)'),
    )
    id = Column(Integer, primary_key=True)
    card_id = Column(Integer, ForeignKey('FifaCard.id'),  nullable=False)
    number = Column(INTEGER(unsigned=True),  nullable=True)
    position_id = Column(Integer, ForeignKey('Position.id'),  nullable=True)
    
    def __repr__(self):
        return "<CardInNationalTeam(id='{0}', card_id={1}, number={2}, position_id={4}}".format(self.id, self.card_id,\
       self.number,  self.position_id)

In [46]:
Base.metadata.create_all(engine)

# Filling database

### Common tables

In [47]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()
session.rollback()

### Clubs

In [48]:
for club in set(clubs.tolist()):
    session.add(Club(name=club))

In [49]:
session.query(Club).limit(5).all()

[<Club(id='1', name=CD Antofagasta,
 <Club(id='2', name=Independiente Santa Fe,
 <Club(id='3', name=Cerezo Osaka,
 <Club(id='4', name=Sassuolo,
 <Club(id='5', name=Atlético Tucumán]

### Nationalities

In [50]:
for national_team in set(national_teams.tolist()):
    session.add(Nationality(name=national_team))

In [51]:
session.query(Nationality).limit(5).all()

[<Nationality(id='1', name=Lebanon,
 <Nationality(id='2', name=Thailand,
 <Nationality(id='3', name=China PR,
 <Nationality(id='4', name=Bolivia,
 <Nationality(id='5', name=Bulgaria]

### Body Types

In [52]:
for body_type in set(body_types.tolist()):
    session.add(BodyType(body_type=body_type))

In [53]:
session.query(BodyType).limit(5).all()

[<BodyType(id='1', body_type=Stocky,
 <BodyType(id='2', body_type=Normal,
 <BodyType(id='3', body_type=Neymar,
 <BodyType(id='4', body_type=Akinfenwa,
 <BodyType(id='5', body_type=Courtois]

### Work Rates

In [54]:
for work_rate in set(work_rates.tolist()):
    session.add(WorkRate(work_rate=work_rate))

In [55]:
session.query(WorkRate).limit(5).all()

[<WorkRate(id='1', work_rate=Medium/Low,
 <WorkRate(id='2', work_rate=Low/High,
 <WorkRate(id='3', work_rate=Low/Medium,
 <WorkRate(id='4', work_rate=High/High,
 <WorkRate(id='5', work_rate=Medium/High]

### Traits

In [56]:
for trait in set(player_traits):
    session.add(Trait(trait=trait))

In [57]:
session.query(Trait).limit(5).all()

[<Trait(id='1', trait=Diver,
 <Trait(id='2', trait=Injury Prone,
 <Trait(id='3', trait=Dives Into Tackles (CPU AI Only),
 <Trait(id='4', trait=Avoids Using Weaker Foot,
 <Trait(id='5', trait=Skilled Dribbling]

Tags

In [58]:
for tag in set(player_tags):
    session.add(Tag(tag=tag))


In [59]:
session.query(Tag).limit(5).all()

[<Tag(id='1', tag=Crosser,
 <Tag(id='2', tag=Aerial Threat,
 <Tag(id='3', tag=Poacher,
 <Tag(id='4', tag=Complete Midfielder,
 <Tag(id='5', tag=Tackling]

In [60]:
for position in set(player_positions):
    session.add(Position(position=position))

In [61]:
session.query(Position).limit(5).all()

[<Position(id='1', position=LM,
 <Position(id='2', position=LAM,
 <Position(id='3', position=ST,
 <Position(id='4', position=RWB,
 <Position(id='5', position=RDM]

In [62]:
session.commit()

### FifaCard 

### Some utils for adding object to FifaCardTable

In [63]:
from sqlalchemy import sql

def add_to_session_and_get_id(session, db_object):
    session.add(db_object)
    session.flush()
    session.refresh(db_object)
    return db_object.id

def is_null(db_object):
    return str(db_object) == "nan"

def get_null_or_val(db_object):
    return sql.null() if is_null(db_object) else db_object


### Detailed Stats

In [64]:
def add_attacking_stats(session, df_tuple):
    crossing = df_tuple.attacking_crossing
    finishing = df_tuple.attacking_finishing
    heading = df_tuple.attacking_heading_accuracy
    short_pass = df_tuple.attacking_short_passing
    volleys  = df_tuple.attacking_volleys
    
    attacking_stats = AttackingStats(crossing=crossing,
                  finishing=finishing,
                  heading=heading,
                  short_pass=short_pass,
                  volleys=volleys)
    
    return add_to_session_and_get_id(session,attacking_stats)
    
def add_skills_stats(session, df_tuple):
    dribbling = df_tuple.skill_dribbling
    curve = df_tuple.skill_curve
    fk_accuracy = df_tuple.skill_fk_accuracy
    long_passing = df_tuple.skill_long_passing
    ball_control  = df_tuple.skill_ball_control
    
    skill_stats = SkillStats(dribbling=dribbling,
              curve=curve,
              fk_accuracy=fk_accuracy,
              long_passing=long_passing,
              ball_control=ball_control)
    
    return add_to_session_and_get_id(session, skill_stats)
    
def add_movement_stats(session, df_tuple):
    acceleration = df_tuple.movement_acceleration
    sprint = df_tuple.movement_sprint_speed
    agility = df_tuple.movement_agility
    reactions = df_tuple.movement_reactions
    balance = df_tuple.movement_balance
    
    movement_stats = MovementStats(acceleration=acceleration,
                                  sprint=sprint,
                                  agility=agility,
                                  reactions=reactions,
                                  balance=balance)
    
    return add_to_session_and_get_id(session, movement_stats)
   
def add_power_stats(session, df_tuple):
    shot_power = df_tuple.power_shot_power
    jumping = df_tuple.power_jumping
    stamina = df_tuple.power_stamina
    strength = df_tuple.power_strength
    long_shots = df_tuple.power_long_shots
    
    power_stats = PowerStats(shot=shot_power,
                            jumping=jumping,
                            stamina=stamina,
                            strength=strength,
                            long_shot=long_shots)
    
    return add_to_session_and_get_id(session, power_stats)

def add_mentality_stats(session, df_tuple):
    
    aggression = df_tuple.mentality_aggression
    interceptions = df_tuple.mentality_interceptions
    positioning = df_tuple.mentality_positioning
    vision = df_tuple.mentality_vision
    penalties = df_tuple.mentality_penalties
    composure = df_tuple.mentality_composure
    
    mentality_stats = MentalityStats(aggression=aggression,
                                    interceptions=interceptions,
                                    positioning=positioning,
                                    vision=vision,
                                    penalties=penalties,
                                    composure=composure)
    
    return add_to_session_and_get_id(session, mentality_stats)

def add_def_stats(session, df_tuple):
    
    marking = df_tuple.defending_marking
    standing_tackle = df_tuple.defending_standing_tackle
    sliding_tackle = df_tuple.defending_sliding_tackle
    
    def_stats = DefendingStats(marking=marking,
                              standing_tackle=standing_tackle,
                              sliding_tackle=sliding_tackle)
    
    return add_to_session_and_get_id(session, def_stats)

def add_goalkeeping_stats(session, df_tuple):
    
    diving = df_tuple.goalkeeping_diving
    handling = df_tuple.goalkeeping_handling
    kicking = df_tuple.goalkeeping_kicking
    positioning = df_tuple.goalkeeping_positioning
    reflexes = df_tuple.goalkeeping_reflexes
    
    goalkeeping_stats = GoalkeepingStats(diving=diving,
                                        handling=handling,
                                        kicking=kicking,
                                        positioning=positioning,
                                        reflexes=reflexes)
    
    return add_to_session_and_get_id(session, goalkeeping_stats)
    


def add_detailed_stats(session, df_tuple):
    attacking_stats_id = add_attacking_stats(session, df_tuple)
    skills_stats_id = add_skills_stats(session, df_tuple)
    movement_stats_id = add_movement_stats(session, df_tuple)
    power_stats_id = add_power_stats(session, df_tuple)
    mentality_stats_id = add_mentality_stats(session, df_tuple)
    def_stats_id = add_def_stats(session, df_tuple)
    goalkeeping_stats_id = add_goalkeeping_stats(session, df_tuple)
    
    detailed_stats = DetailedStats(attacking_id=attacking_stats_id,
                                  skills_id=skills_stats_id,
                                  movement_id=movement_stats_id,
                                  power_id=power_stats_id,
                                  mentality_id=mentality_stats_id,
                                  defending_id=def_stats_id,
                                  goalkeeping_id=goalkeeping_stats_id)
    
    return add_to_session_and_get_id(session, detailed_stats)

### Overall stats

In [65]:
import numpy as np

def add_field_player_stats(session, df_tuple):
    pace = df_tuple.pace
    shooting = df_tuple.shooting
    passing = df_tuple.passing
    dribbling = df_tuple.dribbling
    defending  = df_tuple.defending
    physic  = df_tuple.physic
        
    filed_player_stats = FieldPlayerStats(pace=pace,
                    shooting=shooting,
                    passing=passing,
                    dribbling=dribbling,
                    defending = defending,
                    physic=physic)
    
    return add_to_session_and_get_id(session, filed_player_stats)

def add_goalkeeper_stats(session, df_tuple):
    diving = df_tuple.gk_diving
    handling = df_tuple.gk_handling
    kicking = df_tuple.gk_kicking
    reflexes = df_tuple.gk_reflexes
    speed = df_tuple.gk_speed
    positioning  = df_tuple.gk_positioning
        
    goalkeeper_stats = GoalkeeperStats(diving=diving,
                    handling=handling,
                    kicking=kicking,
                    reflexes=reflexes,
                    speed = speed,
                    positioning=positioning)
    
    
    return add_to_session_and_get_id(session, goalkeeper_stats)

def is_goalkeeper(df_tuple):
    return is_null(df_tuple.pace)

def add_overall_stats(session, df_tuple):
    overall = df_tuple.overall
    potential = df_tuple.potential
    overall_id = 0
    if is_goalkeeper(df_tuple):
        goalkeeper_id = add_goalkeeper_stats(session, df_tuple)
        overall_id = add_to_session_and_get_id(session,OverallStats(overall=overall,
                    potential = potential,
                    goalkeeper_id =goalkeeper_id ))
    else:
        field_player_id = add_field_player_stats(session,df_tuple)
        overall_id = add_to_session_and_get_id(session,OverallStats(overall=overall,
                    potential = potential,
                    field_player_id =field_player_id ))
        
    return overall_id

### Fifa Card

In [66]:
from sqlalchemy import sql

def find_body_type_id(session, body_type):
    return session.query(BodyType.id).filter(BodyType.body_type == body_type).all()[0][0]

def find_work_rate_id(session, work_rate):
    return session.query(WorkRate.id).filter(WorkRate.work_rate == work_rate).all()[0][0]

def find_position_id(session, position):
    return session.query(Position.id).filter(Position.position == position).all()[0][0]

def find_club_id(session, club_name):
    return session.query(Club.id).filter(Club.name == club_name).all()[0][0]

def find_national_team_id(session, national_team_name):
    return session.query(Nationality.id).filter(Nationality.name == national_team_name).all()[0][0]

def add_personal_physical(session, df_tuple):
    
    body_type_id = find_body_type_id(session, df_tuple.body_type)
    height_cm = df_tuple.height_cm
    weight_kg= df_tuple.weight_kg
    
    physical = PersonalPhysical(height= height_cm,
                               weight=weight_kg,
                               body_type_id = body_type_id)
    
    return add_to_session_and_get_id(session, physical)

def add_personal_info(session, df_tuple):
    short_name =df_tuple.short_name
    long_name =df_tuple.long_name
    dob = df_tuple.dob
    persoanl_physical_id = add_personal_physical(session, df_tuple)
    nationality = df_tuple.nationality
    

    personal_info = PersonalInfo(short_name=short_name,
                       long_name=long_name,
                       dob=dob,
                       nationality=nationality)

    return add_to_session_and_get_id(session, personal_info)
    
def add_stars(session, df_tuple):
    
    international_reputation = df_tuple.international_reputation    
    weak_foot = df_tuple.weak_foot
    skill_moves = skill_moves.weak_foot
    
    stars = Stars(skills = skill_moves,
                 reputation = international_reputation,
                 weak_foot = weak_foot)
    
    return add_to_session_and_get_id(session, stars)

def better_foot_to_enum(better_foot):
    return BetterFoot.LEFT if  better_foot == "Left" else BetterFoot.RIGHT 

def real_face_to_boolean(real_face):
    return True if real_face == "Yes" else False
        
def add_fifa_card(session, df_tuple):
    personal_info_id = add_personal_info(session, df_tuple)
    personal_physical_id = add_personal_physical(session, df_tuple)
    real_face = real_face_to_boolean(df_tuple.real_face)
    better_foot = better_foot_to_enum(df_tuple.preferred_foot)
    overall_stats_id = add_overall_stats(session, df_tuple)
    detailed_stats_id = add_detailed_stats(session,df_tuple)
    work_rate_id = find_work_rate_id(session, df_tuple.work_rate)
    value_eur = df_tuple.value_eur
    nationality_id  =find_national_team_id(session, df_tuple.nationality)
    
    fifa_card = FifaCard(personal_info_id=personal_info_id,
                         personal_physical_id= personal_physical_id,
                         work_rate_id = work_rate_id,
                         overall_stats_id=overall_stats_id,
                         detailed_stats_id= detailed_stats_id,
                         real_face=real_face,
                         better_foot=better_foot,
                         value_eur=value_eur,
                         nationality_id = nationality_id)
    
    return add_to_session_and_get_id(session, fifa_card)

def add_player_to_position(session, player_positions, fifa_card_id):
    player_positions = player_positions.split(',')
    positions = [position.strip() for position in player_positions]
    for position in positions:
        position_id = find_position_id(session, position)    
        add_to_session_and_get_id(session, PositionToFifaCard(card_id = fifa_card_id,
                          position_id=position_id))
    
    
def add_contract_details(session, df_tuple):
    wag_eur = df_tuple.wage_eur
    release_eur = get_null_or_val(df_tuple.release_clause_eur)
    club_loaned_id = sql.null()
    valid_until = df_tuple.contract_valid_until
    if not is_null(df_tuple.loaned_from):
        club_loaned_id = find_club_id(session, df_tuple.loaned_from)
    
    details = ContractDetails(wag_eur=wag_eur,
                             release_eur=release_eur,
                             loaded_from=club_loaned_id,
                             valid_until=valid_until)
    
    return add_to_session_and_get_id(session, details)
    
def card_has_club(df_tuple):
    return is_null(df_tuple.wage_eur)

def add_player_in_club(session, df_tuple, fifa_card_id):
    
    if card_has_club(df_tuple): # Skip the player who not plaing in any club right now 
        
        position_id = sql.null()
        if not is_null(df_tuple.team_position):
            position_id = find_position_id(session, df_tuple.team_position)
            
        number = get_null_or_val(df_tuple.team_jersey_number)
        joined = get_null_or_val(df_tuple.joined) 
        contract_details_id = add_contract_details(session, df_tuple)
        club_id = find_club_id(session, df_tuple.club)
        
        card_in_club = CardInClub(card_id = fifa_card_id,
                                 position=position_id,
                                 number=number,
                                 joined=joined,
                                 club_id = club_id,
                                 contract_details_id = contract_details_id )
        
        add_to_session_and_get_id(session, card_in_club)

def card_represent_national_team(df_tuple):
    return not is_null(df_tuple.nation_jersey_number)

def add_player_in_national_team(session, df_tuple, fifa_card_id):
    
    if card_represent_national_team(df_tuple): # Skip the player who has not been representing any country so far. 
            
        position_id = sql.null()
        number = get_null_or_val(df_tuple.team_jersey_number)
        
        if not is_null(df_tuple.team_position):
            position_id = find_position_id(session, df_tuple.team_position)
                    
        card_in_national_team = CardInNationalTeam(card_id = fifa_card_id,
                                 number=number,
                                 position_id=position_id)
        
        return add_to_session_and_get_id(session, card_in_national_team)

### Tags and Traits To Players

In [70]:
import numpy as np

def find_tag_id(session, tag):
    return session.query(Tag.id).filter(Tag.tag == tag)[0][0]

def card_has_tags(df_tuple):
    return not str(df_tuple.player_tags) == "nan" 
    
def add_tag_to_fifa_card_id(session, df_tuple, card_id):
    if card_has_tags(df_tuple):
        tags = ["".join(tag.strip()[1:]) for tag in df_tuple.player_tags.split(',')]
        for tag in tags:
            tag_id = find_tag_id(session, tag)    
            add_to_session_and_get_id(session, TagToFifaCard(card_id = card_id,
                              tag_id=tag_id))
            
def card_has_traits(df_tuple):
    return not is_null(df_tuple.player_traits) 

def find_trait_id(session, trait):
    return session.query(Trait.id).filter(Trait.trait == trait)[0][0]

def add_trait_to_fifa_card_id(session, df_tuple, card_id):
    if card_has_traits(df_tuple):
        traits = [trait.strip() for trait in df_tuple.player_traits.split(',')]
        for trait in traits:
            trait_id = find_trait_id(session, trait)    
            add_to_session_and_get_id(session, TraitToFifaCard(card_id = card_id,
                              trait_id=trait_id))

# Add all stuff

In [71]:
def add_all_stuff(session, df_tuple):
    fifa_card_id = add_fifa_card(session, df_tuple)
    add_trait_to_fifa_card_id(session, df_tuple, fifa_card_id)
    add_tag_to_fifa_card_id(session, df_tuple, fifa_card_id)
    add_player_in_club(session, df_tuple, fifa_card_id)
    add_player_in_national_team(session, df_tuple, fifa_card_id)


In [72]:
for df_tuple in df.itertuples():
    add_all_stuff(session, df_tuple)

In [73]:
session.query(FifaCard).limit(10).all()

[<FifaCard(id='1', personal_info_id=1, overall_stats_id=1        detailed_stats_id=1, work_rate_id=1, real_face=True, better_foot=BetterFoot.LEFT, personal_physical_id=2, value_eur=95500000, nationality_id=2,
 <FifaCard(id='2', personal_info_id=2, overall_stats_id=2        detailed_stats_id=2, work_rate_id=7, real_face=True, better_foot=BetterFoot.RIGHT, personal_physical_id=4, value_eur=58500000, nationality_id=4,
 <FifaCard(id='3', personal_info_id=3, overall_stats_id=3        detailed_stats_id=3, work_rate_id=8, real_face=True, better_foot=BetterFoot.RIGHT, personal_physical_id=6, value_eur=105500000, nationality_id=6,
 <FifaCard(id='4', personal_info_id=4, overall_stats_id=4        detailed_stats_id=4, work_rate_id=6, real_face=True, better_foot=BetterFoot.RIGHT, personal_physical_id=8, value_eur=77500000, nationality_id=8,
 <FifaCard(id='5', personal_info_id=5, overall_stats_id=5        detailed_stats_id=5, work_rate_id=8, real_face=True, better_foot=BetterFoot.RIGHT, personal_phy

In [74]:
session.commit()